In [1]:
!pip install pandas pyarrow selenium

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import json
import time

import pandas as pd
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [3]:
file_path = './final_data/surnames/surname.csv'

HOMEPAGE = "https://albo.alboweb-fnofi.net/registry/search"

data = []

index_pointer = 0

try:
    with open('fnofi_last_index.txt', 'r') as file:
        content = file.read()
        index_pointer = int(content)
except FileNotFoundError:
    print("File 'fnofi_last_index.txt' not found. Using default index_pointer value.")

name_of_file = f"fnofi_data_{index_pointer}"

In [4]:
translation_professions_into_degrees = {
    'Assistente Sanitario': 'Assistenza Sanitaria',
    'Tecnico sanitario di laboratorio biomedico': 'Tecniche di Laboratorio Biomedico',
    'Tecnico audiometrista': 'Tecniche Audioprotesiche',
    'Tecnico audioprotesista': 'Tecniche Audioprotesiche',
    'Tecnico ortopedico': 'Tecniche Ortopediche',
    'Dietista': 'Dietistica',
    'Tecnico di neurofisiopatologia': 'Tecniche di Neurofisiopatologia',
    'Tecnico fisiopat. cardiocirc. e perfusione cardiov.': 'Tecniche della Fisiopatologia Cardiocircolatoria e Perfusione Cardiovascolare',
    'Igienista dentale': 'Igiene Dentale',
    'Logopedista': 'Logopedia',
    'Podologo': 'Podologia',
    'Ortottista e assistente di oftalmologia': 'Ortottica e Assistenza Oftalmologica',
    "Terapista della neuro e psicomotricità dell'età evolutiva": "Terapia della Neuro e Psicomotricità dell'Età Evolutiva",
    'Tecnico della riabilitazione psichiatrica': 'Tecniche della Riabilitazione Psichiatrica',
    'Terapista occupazionale': 'Terapia Occupazionale',
    'Educatore professionale': 'Educazione Professionale',
    'Tecnico prevenzione ambiente e luoghi di lavoro': "Tecniche della Prevenzione nell'Ambiente e nei Luoghi di Lavoro",
    'Elenco speciale ad esaurimento: Tecnico sanitario di laboratorio biomedico': 'Tecniche di Laboratorio Biomedico',
    'Elenco speciale ad esaurimento: Tecnico audiometrista': 'Tecniche Audioprotesiche',
    'Elenco speciale ad esaurimento: Tecnico audioprotesista': 'Tecniche Audioprotesiche',
    'Elenco speciale ad esaurimento: Tecnico ortopedico': 'Tecniche Ortopediche',
    'Elenco speciale ad esaurimento: Dietista': 'Dietistica',
    'Elenco speciale ad esaurimento: Tecnico di neurofisiopatologia': 'Tecniche di Neurofisiopatologia',
    'Elenco speciale ad esaurimento: Tecnico della fisiopatologia cardiocircolatoria e perfusione cardiovascolare': 'Tecniche della Fisiopatologia Cardiocircolatoria e Perfusione Cardiovascolare',
    'Elenco speciale ad esaurimento: Igienista dentale': 'Igiene Dentale',
    'Elenco speciale ad esaurimento: Logopedista': 'Logopedia',
    'Elenco speciale ad esaurimento: Podologo': 'Podologia',
    'Elenco speciale ad esaurimento: Ortottista e assistente di oftalmologia': 'Ortottica e Assistenza Oftalmologica',
    "Elenco speciale ad esaurimento: Terapista della neuro e psicomotricità dell'età evolutiva": "Terapia della Neuro e Psicomotricità dell'Età Evolutiva",
    'Elenco speciale ad esaurimento: Tecnico della riabilitazione psichiatrica': 'Tecniche della Riabilitazione Psichiatrica',
    'Elenco speciale ad esaurimento: Terapista occupazionale': 'Terapia Occupazionale',
    'Elenco speciale ad esaurimento: Educatore professionale': 'Educazione Professionale',
    "Elenco speciale ad esaurimento: Tecnico della prevenzione nell'ambiente e nei luoghi di lavoro": "Tecniche della Prevenzione nell'Ambiente e nei Luoghi di Lavoro",
    'Elenco speciale ad esaurimento: Massofisioterapista': 'Diploma in Massofisioterapia',
    'Società tra professionisti': 'Società tra professionisti'
}


In [5]:
def get_data(url, df):
    browser_options = ChromeOptions()
    driver = Chrome(options=browser_options)

    driver.get(url)

    wait = WebDriverWait(driver, 10)
    global index_pointer
    
    try:
        for outer_index, row in df.iloc[index_pointer:].iterrows():
            sure_name_input = row["surname"]
            index_pointer += 1
            print({"name": sure_name_input, "pointer": index_pointer, "index": outer_index,"total Data":len(data)})
    
    
            sure_name = wait.until(EC.presence_of_element_located((By.ID, "last_name")))
            search = driver.find_element(By.ID, "saveSearch")
    
            sure_name.send_keys(sure_name_input)
            search.click()
    
            WebDriverWait(driver, 10).until(
                lambda driver: driver.execute_script('return document.readyState') == 'complete')
    
            time.sleep(1)
            # table_data = wait.until(EC.presence_of_element_located((By.ID, "awTable")))
            table_data = driver.execute_script("""
            let table = $('#awTable').DataTable();
            let data = table.rows().data();
            let finalData = Object.entries(data)
            .filter(([key, value]) => !isNaN(Number(key)))
            .reduce((acc, [key, value]) => {
                acc[key] = value;
                return acc;
            }, {});;
            return JSON.stringify(finalData);
            """)
    
            parsed_data = json.loads(table_data)
            if len(parsed_data.values())>0:
                for value in parsed_data.values():
                    user = {}
                    user["person_id"] = value["id"]
                    user["surname"] = value["last_name"]
                    user["first_name"] = value["first_name"]
                    user["date_of_birth"] = value["birth_date"]
                    user["birth_place"] = value["birth_place"]
                    user["client_name"] = value["client_name"]
                    full_name_data_string = value['full_name']
                    full_name_data_list = json.loads(full_name_data_string)
                    user["special_register"] = full_name_data_list
        
                    if user:
                        data.append(user)
                        
                    
            driver.refresh()
    except Exception as e:
        print(f"Error Message777: {e}")


In [6]:
def export_data(data, name):
    df = pd.DataFrame(data)
    df.to_csv(f"fnofi_data/{name}.csv", index=False)
    with open(f"fnofi_data/{name}.json", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(df)  # DEBUG

In [7]:
def main():
    try:
        start_time = time.time()
        df = pd.read_csv(file_path)
        get_data(url=HOMEPAGE, df=df)
        end_time = time.time()
        total_time = end_time - start_time
        print(f"DONE Total time taken: {total_time} seconds")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        export_data(data, name_of_file)
        with open('fnofi_last_index.txt', 'w') as file:
            file.write(str(index_pointer))
        print("The last index was", index_pointer)

In [8]:
while True:
    main()

{'name': 'Abbruzzi ', 'pointer': 80, 'index': 79, 'total Data': 0}
{'name': 'Abbruzzo ', 'pointer': 81, 'index': 80, 'total Data': 0}
{'name': 'Abbundo ', 'pointer': 82, 'index': 81, 'total Data': 1}
{'name': "Abbura' ", 'pointer': 83, 'index': 82, 'total Data': 1}
{'name': 'Abburà ', 'pointer': 84, 'index': 83, 'total Data': 1}
{'name': 'Abd Alla ', 'pointer': 85, 'index': 84, 'total Data': 1}
{'name': 'Abdalla ', 'pointer': 86, 'index': 85, 'total Data': 1}
{'name': 'Abdallah ', 'pointer': 87, 'index': 86, 'total Data': 1}
{'name': 'Abdel Rahman ', 'pointer': 88, 'index': 87, 'total Data': 2}
{'name': 'Abdelrahman ', 'pointer': 89, 'index': 88, 'total Data': 2}
{'name': 'Abdo ', 'pointer': 90, 'index': 89, 'total Data': 2}
{'name': 'Abdou ', 'pointer': 91, 'index': 90, 'total Data': 2}
{'name': 'Abdul ', 'pointer': 92, 'index': 91, 'total Data': 2}
{'name': 'Abdullah ', 'pointer': 93, 'index': 92, 'total Data': 2}
{'name': 'Abdullahi ', 'pointer': 94, 'index': 93, 'total Data': 2}
{'

KeyboardInterrupt: 